# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [64]:
## IMPORTS
# core config
from azureml.core import Workspace, Experiment, Environment, Model
# compute config
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

#HD config

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import choice
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.policy import TruncationSelectionPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive import HyperDriveRun
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.core import ScriptRunConfig
from azureml.core import Webservice

import os

import sklearn

import requests
import json

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'EXP_HEART_FAILURE_HD'

experiment=Experiment(ws, experiment_name)


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-145543
Azure region: southcentralus
Subscription id: 5a4ab2ba-6c51-4805-8155-58759ad589d8
Resource group: aml-quickstarts-145543


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.
I am planning to use xgboost model for the hyperparameter tuning.
I have done a baseline modeling on 2 different model types, locally
1- Random Forest, which is very simple, well interpretable ensemble model, hard/impossible to overtrain, works well on non-linear problems
2- xgboost, which is a boosted ensemble model, with a lots of tunable parameters

With RF I was able to get a 0.73 AUC, which is a medium strength model.
With xgboost, using 5 fold CV, but without really running through parameter space locally I was able to achieve 0.88
The catch with the xgboost, that it typically performs well on large dataset, the provided dataset has 300 rows only, so there is a risk of overtraining
So I will need to focus on regularization of the model.

I am using 2 paths for this: an indirect regularization with boost rounds and learning rate and a direct one with gamma and lambdta


Target metric, as the dataset is for classification problem is AUC.

Initially I am using RandomParameterSampling to run through a wider parameter space. 
For this I will use an aggressive erly stopping policy, to save time and money.

As I am using xgboost model, I am not using estimator, I use a run config instead, with an xgboost model

From the learnings I will set up another round 


In [3]:
#As i do not want to wait for the cluster nodes to get activated, I set up all of them active immediately
cpu_cluster_name = "hd-gyula-01"
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("This compute exists")
except ComputeTargetException:
    print("Building my own")
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", min_nodes=4, max_nodes = 4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Building my own
Creating...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
# Initial parameter space with sampler, many parameters, wide space
# https://xgboost.readthedocs.io/en/latest/parameter.html
ps_random = {"n_estimators": choice(50, 100, 150, 200, 300, 500),
              "max_depth": choice(3, 5, 8, 13),
              "min_samples_split": choice(2,3,4)
              }

random_samp = RandomParameterSampling(parameter_space=ps_random)

In [8]:
truncation_selection_policy = TruncationSelectionPolicy(evaluation_interval=1, truncation_percentage=30, delay_evaluation=5)

In [14]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name", env)
        print("Packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-Pytorch1.7-Cuda11-OpenMpi4.1.0-py36
Packages channels:
- anaconda
- pytorch
- conda-forge
dependencies:
- python=3.6.9
- pip>=21.0,<22
- pytorch==1.7.1
- torchvision==0.8.2
- torchaudio==0.7.2
- cudatoolkit=11.0
- nvidia-apex==0.1.0
- pip:
  - matplotlib>=3.3,<3.4
  - psutil>=5.8,<5.9
  - tqdm>=4.59,<4.60
  - pandas>=1.1,<1.2
  - theano>=1.0,<1.1
  - scipy>=1.5,<1.6
  - numpy>=1.10,<1.20
  - azureml-core==1.26.0
  - azureml-defaults==1.26.0
  - azureml-mlflow==1.26.0
  - azureml-telemetry==1.26.0
  - azureml-train-restclients-hyperdrive==1.26.0
  - azureml-train-core==1.26.0
  - tensorboard==2.4.0
  - horovod==0.20.0
  - onnxruntime-gpu>=1.7,<1.8
  - future==0.17.1
name: azureml_a3e3434bd3fec67ad455dfe16747f230

Name AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36
Packages channels:
- anaconda
- conda-forge
dependencies:
- python=3.6.9
- pip>=21.0,<22
- pip:
  - matplotlib>=3.3,<3.4
  - psutil>=5.8,<5.9
  - tqdm>=4.59,<4.60
  - pandas>=1.1,<1.2
  - theano>=1.0,<1.1
  - s

In [16]:
myenv = Environment.get(workspace=ws, name="AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36")

In [50]:
# instead of estimator i am using a run config
src = ScriptRunConfig(source_directory="./script",
                      script="hyper_tuning_rf.py",
                      arguments=['--n_estimators', 10, '--max_depth', 3, '--min_samples_split', 2],
                      compute_target=compute_target,
                      environment=myenv)

In [51]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = truncation_selection_policy

#TODO: Create the different params that you will be using during training
param_sampling = random_samp

#TODO: Create your estimator and hyperdrive config
#estimator = <your estimator here>

hyperdrive_run_config =  HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     primary_metric_name="AUC",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=30,
                                     max_concurrent_runs=4)

In [19]:
if "training" not in os.listdir():
    os.mkdir("./training")

In [52]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [53]:
widget1 = RunDetails(hyperdrive_run)
widget1.show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [56]:
# display properties of the model
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Best run metrics for Random Forest\n", best_run_metrics)

Best run metrics for Random Forest
 {'n_estimators:': 300, 'max_depth:': 8, 'min_samples_split:': 4, 'AUC': 0.7960224375318715}


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [61]:
#TODO: Save the best model
model = best_run.register_model(
    model_name="sklearn-randomforest-hd",
    model_path="./outputs/model.pkl",
    model_framework = Model.Framework.SCIKITLEARN,
    model_framework_version=sklearn.__version__,
    description="Random forest classification model for heart failure analysis",
    tags={'area':'heart-failure', 'type':'classification'}
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [66]:

name = 'mdldply-heartfailure-rf-hd'
scoring_uri = "http://b5f9be12-a7cd-403b-8bf6-0ff0cdb7cd66.southcentralus.azurecontainer.io/score"
key = "dGJiT6vZj1sjkOmkRzlPoMR50a3rvB4O"




In [67]:
service = Webservice(name=name, workspace=ws)
service.update(enable_app_insights=True)

WebserviceException: WebserviceException:
	Message: There is a deployment operation in flight for the Service: mdldply-heartfailure-rf-hd
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "There is a deployment operation in flight for the Service: mdldply-heartfailure-rf-hd"
    }
}

In [62]:
data_json = {
	"data":
	[
		{"age": 75, "anaemia":0, "creatinine_phosphokinase": 582, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 265000, "serum_creatinine": 1.9, "serum_sodium": 130, "sex": 1, "smoking": 0, "time":4}
	]

}

In [68]:



input_data = json.dumps(data_json)
with open("data.json", "w" ) as _f:
    _f.write(input_data)
headers =  {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

{'predict_proba': [[0.026230994152046785, 0.9737690058479536]]}


TODO: In the cell below, print the logs of the web service and delete the service

In [69]:
service = Webservice(name=name, workspace=ws)

# enable application insight
# we have done this from notebook
#service.update(enable_app_insights=True)

logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

2021-05-24T13:51:22,854156500+00:00 - rsyslog/run 
2021-05-24T13:51:22,855172800+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
2021-05-24T13:51:22,954201900+00:00 - nginx/run 
2021-05-24T13:51:22,963577100+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-05-24T13:51:25,345905600+00:00 - iot-server/finish 1 0
2021-05-24T13:51:25,355169000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (65)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 91
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-05-24 13:51:38,456 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-05-24 13:51:38,463 | root | INFO | Starting up request id generator
2021-05-24 13:51:38,463 | root | INFO | Starting up app in